In [4]:
!pip install pulp

#!sudo apt-get install coinor-cbc glpk-utils coinor-clp
!conda install -c conda-forge pulp

     |████████████████████████████████| 40.6MB 98kB/s 
/bin/bash: conda: command not found


In [ ]:
### kod sprawdza czy instalacja się udała
import pulp

print(pulp.__version__)

2.3.1


# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |

Kot ma zjeść minimum 0.24 ug na kilogram masy więc około
0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

Takie zdanie można skleić z części generowanych automatycznie:



In [ ]:
sklejone_produkt_cena = [":".join((produkt, str(cena))) for produkt,cena in zip(["jabłko","banan"],[5,10])]
print(sklejone_produkt_cena)
sklejona_calosc = ", ".join(sklejone_produkt_cena)

print(sklejona_calosc)

['jabłko:5', 'banan:10']
jabłko:5, banan:10


Celem zadanie zrealizowanego podczas zajęć było dobranie odpowiedniego składu karmy dla kota, tak aby zminimalizować koszt produkcji, a także zapewnienie odpowiedzniej ilości wymaganych wartości odżywczych. Zawartość składników odżywczych w poszczególnych produktach zostały przedstawione w tabeli:


| Nazwa      | vitamin b ug/100g | protein (g per g products) | fat | fibre | salt |
|------------|-------------------|----------------------------|-----|-------|------|
| chicken    | 0.3               | 0.1| 0.08| 0.001| 0.002|
| beef       | 2.6               | 0.2| 0.1| 0.005| 0.005|
| mutton     | 2.6               | 0.15| 0.11| 0.003| 0.007|
| rice       | 0                 | 0.0| 0.01| 0,1| 0.002|
| wheat bran | 0                 | 0.04| 0.01| 0.15| 0.008|
| gel        | 0                 | 0.0| 0.0| 0.0| 0.0|
| sardines   | 8.9               | 0.025| 0.011| 0.0| 0.005|
| green peas | 0.15              | 0.005| 0.004| 0.06| 0.0045|

Do składników znajdujących się w opisie zadania zostały dodane sardynki oraz zielony groszek, a wartości składników odżywczych zostały znalezione w internecie. Znana jes także cena poszczególnych produktów. W gotowym produkcie ma znajdować się 8g białka, 6g tłuszczu, 2g błonnika oraaz 0,4g soli, oraz 0,8ug witaminy B.

In [5]:
from pulp import *

Przykładowy program:

In [6]:
prob = LpProblem("The Whiskas Problem",LpMinimize)
x1=LpVariable("ChickenPercent",0,100)
x2=LpVariable("BeefPercent",0,100)
x3=LpVariable("MuttonPercent",0,100,)
prob += 0.013*x1 + 0.008*x2 + 0.01*x3, "Total Cost of Ingredients per can"
prob += 1*x3 >= 1, "PercentagesMutton"
prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.15*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.11*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.003*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.007*x3 <= 0.4, "SaltRequirement"

prob.writeLP("WhiskasModel.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
for v in prob.variables():
    print(v.name, "=", v.varValue)
print("Total Cost of Ingredients per can = ", value(prob.objective))

Status: Optimal
BeefPercent = 65.0
ChickenPercent = 34.0
MuttonPercent = 1.0
Total Cost of Ingredients per can =  0.972


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Program z dodanymi zmiennymi:

In [7]:
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'Sardines', 'Green peas'] #definicja listy składników

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, #słownik określający koszt poszczególnych składników
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'Sardines': 0.01,
         'Green peas': 0.0001,
         }
#następnie zostały zdefiniowane słowniki w których są umieszczone ilości poszczególnych składników odżywczych w składnikach
# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'Sardines': 0.025,
                  'Green peas': 0.005
                  }

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'Sardines': 0.011,
              'Green peas': 0.004,
              }

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'Sardines': 0.0,
                'Green peas': 0.06,
                }

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'Sardines': 0.005,
               'Green peas': 0.0045,
               }
vitaminbPercent = {'CHICKEN': 0.003, 
               'BEEF': 0.026, 
               'MUTTON': 0.026, 
               'RICE': 0.000, 
               'WHEAT': 0.000, 
               'GEL': 0.000,
               'Sardines': 0.001,
               'Green peas': 0.0015,
               }

prob = LpProblem("The Whiskas Problem", LpMinimize) #zdefiniowanie problemu, minimalizacja kosztów
ingredient_vars = LpVariable.dicts("Ingr",Ingredients,0) 
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can" #obliczenie sumy kosztów poszczególnych użytych składników
# The five constraints are added to 'prob'
prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum" #suma zastosowanych składników musi dać 100% produktu
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement" #ograniczenia wynikające z określonych ilości poszczególnych składników odżywczych w gotowym produkcie
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement" 
prob += lpSum([vitaminbPercent[i] * ingredient_vars[i] for i in Ingredients]) == 0.8, "VitaminbRequirement"             
prob.writeLP("WhiskasModel.lp")
prob.solve()
print("Status:", LpStatus[prob.status])
print("Końcowa wartość 1 puszki wynosi:", value(prob.objective))
print("I zawiera:")
for v in prob.variables():
    sklejone_produkt_cena = [":".join((v.name, str(v.varValue)))]
    sklejona_calosc = ", ".join(sklejone_produkt_cena)
    print(sklejona_calosc)


  



Status: Optimal
Końcowa wartość 1 puszki wynosi: 0.7622015741000001
I zawiera:
Ingr_BEEF:23.985476
Ingr_CHICKEN:43.487668
Ingr_GEL:1.9171072
Ingr_Green_peas:30.609749
Ingr_MUTTON:0.0
Ingr_RICE:0.0
Ingr_Sardines:0.0
Ingr_WHEAT:0.0


/usr/local/lib/python3.6/dist-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Otrzymaliśmy optymalny produkt pod względem minimalizacji ceny produkcji. Karma spełnia wszystkie założenia jeśli chodzi o zawartość poszczególnych składników odżywczych. W końcowym składzie znajduje się dodany produkt - zielonmy groszek. Jednak używając ceny znalezionej w internecie groszek nie bbyłby używany w produkcji karmy. Występowały w niej tańsze produkty. Cena groszku na potrzeby zadania została zmniejszona. Sprawiając, że groszek jest tańszy został on dodany do składu końcowego produktu. Świadczy to o tym, że cena poszczególnego produktu jest ważnym czynnnikem wpływającym na to jakich składników powinniśmy użyć w celu zminimalizowania kosztów produkcji.